Pykeen environment setup

In [13]:
! pip install git+https://github.com/pykeen/pykeen.git


  Cloning https://github.com/pykeen/pykeen.git to /tmp/pip-req-build-xs_v8j6j
  Running command git clone --filter=blob:none --quiet https://github.com/pykeen/pykeen.git /tmp/pip-req-build-xs_v8j6j
  Resolved https://github.com/pykeen/pykeen.git to commit 9b9b59fbc08b58a8d37e1b7426d45576ff980d16
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [14]:
! python -c "import pykeen" || pip install git+https://github.com/pykeen/pykeen.git
import pykeen
pykeen.env()

Key,Value
OS,posix
Platform,Linux
Release,6.1.58+
Time,Sun Apr 21 06:54:51 2024
Python,3.10.12
PyKEEN,1.10.3-dev
PyKEEN Hash,UNHASHED
PyKEEN Branch,
PyTorch,2.2.1+cu121
CUDA Available?,true


Training the model

In [15]:
from pykeen.pipeline import pipeline
pipeline_result = pipeline(
    dataset = 'Nations',
    model='TransE', #knowledge graph embedding model
    training_loop='sLCWA',
    negative_sampler='basic',
    evaluator='RankBasedEvaluator',
    stopper='early',
)
pipeline_result.save_to_directory('nations_transe')

INFO:pykeen.datasets.utils:Loading cached preprocessed dataset from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/training
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/testing
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/validation
INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: /root/.data/pykeen/checkpoints/best-model-weights-ac2a2ade-5e05-4a80-adb8-b53612b5c331.pt


Training epochs on cuda:0:   0%|          | 0/5 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/201 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.09s seconds
INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=14, num_relations=55, create_inverse_triples=False, num_triples=1592, path="/usr/local/lib/python3.10/dist-packages/pykeen/datasets/nations/train.txt") to file:///content/nations_transe/training_triples
INFO:pykeen.pipeline.api:Saved to directory: /content/nations_transe


In [16]:
import torch

my_pykeen_model = torch.load('/content/nations_transe/trained_model.pkl')

In [17]:
from typing import List

model = pipeline_result.model

entity_representation_modules: List['pykeen.nn.Representation'] = model.entity_representations
relation_representation_modules: List['pykeen.nn.Representation'] = model.relation_representations

In [18]:
entity_embeddings: pykeen.nn.Embedding = entity_representation_modules[0]
relation_embeddings: pykeen.nn.Embedding = relation_representation_modules[0]

In [19]:
entity_embedding_tensor: torch.FloatTensor = entity_embeddings()
relation_embedding_tensor: torch.FloatTensor = relation_embeddings()

In [20]:
print(entity_embedding_tensor)

tensor([[-1.8213e-01,  1.1843e-01,  1.0015e-01, -8.2214e-03, -1.8195e-01,
         -7.4911e-02,  2.0709e-01, -2.4609e-01, -2.1374e-01, -1.5577e-01,
         -2.2287e-01,  1.2000e-01,  2.0599e-01, -1.8078e-01, -3.1419e-02,
         -3.7488e-02,  5.7789e-02,  1.9122e-01,  4.6935e-02,  1.9774e-01,
          3.2501e-02, -2.1002e-01, -1.9238e-01,  4.6722e-02,  2.5095e-01,
         -6.2082e-02,  1.6757e-03, -1.2235e-01,  1.2628e-02,  3.1990e-02,
          2.2529e-01, -7.6833e-02, -8.6180e-02,  1.1070e-01, -2.0814e-01,
         -9.9684e-02, -1.0487e-01,  1.2861e-01, -1.3750e-01,  1.3940e-01,
          7.0146e-02,  1.9846e-01,  6.8346e-02,  1.3628e-01,  2.3815e-01,
          2.1184e-02,  3.4012e-02,  1.1099e-01,  6.9399e-02,  4.4560e-02],
        [-3.6336e-03,  1.6445e-01, -1.4882e-01, -1.6508e-01, -1.3079e-01,
          1.8787e-01,  1.3697e-01,  7.7608e-02,  2.4948e-02,  2.2706e-01,
         -5.1075e-02,  3.0319e-04,  2.2474e-01, -5.4696e-02,  7.2095e-02,
         -2.0848e-01,  1.4867e-01, -1

Using FBK15

In [38]:
from pykeen.datasets.inductive.ilp_teru import InductiveFB15k237
from pykeen.models.inductive import InductiveNodePieceGNN
from pykeen.training import SLCWATrainingLoop
from pykeen.evaluation.rank_based_evaluator import SampledRankBasedEvaluator
from pykeen.stoppers import EarlyStopper

from torch.optim import Adam

dataset = InductiveFB15k237(version="v1", create_inverse_triples=True)

model = InductiveNodePieceGNN(
    triples_factory=dataset.transductive_training,  # training factory, will be also used for a GNN
    inference_factory=dataset.inductive_inference,  # inference factory, will be used for a GNN
    num_tokens=12,  # length of a node hash - how many unique relations per node will be used
    aggregation="mlp",  # aggregation function, defaults to an MLP, can be any PyTorch function
    loss=NSSALoss(margin=15),  # dummy loss
    random_seed=42,
    gnn_encoder=None,  # defaults to a 2-layer CompGCN with DistMult composition function
)

optimizer = Adam(params=model.parameters(), lr=0.0005)

training_loop = SLCWATrainingLoop(
    triples_factory=dataset.transductive_training,  # training triples
    model=model,
    optimizer=optimizer,
    negative_sampler_kwargs=dict(num_negs_per_pos=32),
    mode="training",   # necessary to specify for the inductive mode - training has its own set of nodes
)

# Validation and Test evaluators use a restricted protocol ranking against 50 random negatives
valid_evaluator = SampledRankBasedEvaluator(
    mode="validation",   # necessary to specify for the inductive mode - this will use inference nodes
    evaluation_factory=dataset.inductive_validation,  # validation triples to predict
    additional_filter_triples=dataset.inductive_inference.mapped_triples,   # filter out true inference triples
)

# According to the original code
# https://github.com/kkteru/grail/blob/2a3dffa719518e7e6250e355a2fb37cd932de91e/test_ranking.py#L526-L529
# test filtering uses only the inductive_inference split and does not include inductive_validation triples
# If you use the full RankBasedEvaluator, both inductive_inference and inductive_validation triples
# must be added to the additional_filter_triples
test_evaluator = SampledRankBasedEvaluator(
    mode="testing",   # necessary to specify for the inductive mode - this will use inference nodes
    evaluation_factory=dataset.inductive_testing,  # test triples to predict
    additional_filter_triples=dataset.inductive_inference.mapped_triples,   # filter out true inference triples
)

early_stopper = EarlyStopper(
    model=model,
    training_triples_factory=dataset.inductive_inference,
    evaluation_triples_factory=dataset.inductive_validation,
    frequency=1,
    patience=100000,  # for test reasons, turn it off
    result_tracker=None,
    evaluation_batch_size=256,
    evaluator=valid_evaluator,
)

# Training starts here
training_loop.train(
    triples_factory=dataset.transductive_training,
    stopper=early_stopper,
    num_epochs=100,
)

# Test evaluation
result = test_evaluator.evaluate(
    model=model,
    mapped_triples=dataset.inductive_testing.mapped_triples,
    additional_filter_triples=dataset.inductive_inference.mapped_triples,
    batch_size=256,
)

# print final results
print(result.to_flat_dict())

sampling:   0%|          | 0.00/1.59k [00:00<?, ?it/s]

sampling:   0%|          | 0.00/1.09k [00:00<?, ?it/s]

INFO:pykeen.evaluation.rank_based_evaluator:Sampling 50 negatives for each of the 206 evaluation triples.
INFO:pykeen.evaluation.rank_based_evaluator:Sampling 50 negatives for each of the 205 evaluation triples.
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: /root/.data/pykeen/checkpoints/best-model-weights-379fdde4-6eb8-41b6-8572-b7cb81a596fc.pt
INFO:pykeen.training.training_loop:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value '{batch_size}'


Training epochs on cpu:   0%|          | 0/100 [00:00<?, ?epoch/s]

INFO:pykeen.triples.triples_factory:Creating inverse triples.
INFO:pykeen.training.training_loop:Dropping last (incomplete) batch each epoch (1/33 (3.03%) batches).


...

Evaluating on cpu:   0%|          | 0.00/206 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.30s seconds


Evaluating on cpu:   0%|          | 0.00/205 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.31s seconds


{'head.optimistic.median_rank': 8.0, 'tail.optimistic.median_rank': 5.0, 'both.optimistic.median_rank': 7.0, 'head.realistic.median_rank': 8.0, 'tail.realistic.median_rank': 5.0, 'both.realistic.median_rank': 7.0, 'head.pessimistic.median_rank': 8.0, 'tail.pessimistic.median_rank': 5.0, 'both.pessimistic.median_rank': 7.0, 'head.optimistic.adjusted_geometric_mean_rank_index': 0.6432026947146352, 'tail.optimistic.adjusted_geometric_mean_rank_index': 0.7736259097197293, 'both.optimistic.adjusted_geometric_mean_rank_index': 0.714358178515274, 'head.realistic.adjusted_geometric_mean_rank_index': 0.6431165540094395, 'tail.realistic.adjusted_geometric_mean_rank_index': 0.773604824124855, 'both.realistic.adjusted_geometric_mean_rank_index': 0.7143098075131133, 'head.pessimistic.adjusted_geometric_mean_rank_index': 0.6430323818971901, 'tail.pessimistic.adjusted_geometric_mean_rank_index': 0.7735839810653973, 'both.pessimistic.adjusted_geometric_mean_rank_index': 0.714262385942553, 'head.optimi